In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import time
import numpy as np
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

os.chdir('/content/drive/MyDrive/MeetfreshCV Group 3')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from IPython.display import Image, display

listOfImageNames = ['/path/to/images/1.png',
                    '/path/to/images/2.png']

for imageName in listOfImageNames:
    display(Image(filename=imageName))


In [ ]:
# split train, valid, test
# !pip install split-folders
# import splitfolders
# splitfolders.ratio('data_aug', output = 'data_model', seed = 2022, ratio = (.7, 0.2, 0.1)) 

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

BATCH_SIZE = 500
IMG_SIZE = 100
IMG_DIR = 'data_model'

train = ImageDataGenerator(rescale = 1./255)
train_dataset = train.flow_from_directory(
    directory = IMG_DIR + '/train',
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
    shuffle = True)

valid = ImageDataGenerator(rescale = 1./255)
valid_dataset = valid.flow_from_directory(
    directory = IMG_DIR + '/val',
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
    shuffle = False)

test = ImageDataGenerator(rescale = 1./255)
test_dataset = test.flow_from_directory(
    directory = IMG_DIR + '/test',
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
    shuffle = False)

Found 3874 images belonging to 14 classes.
Found 1102 images belonging to 14 classes.
Found 567 images belonging to 14 classes.


In [ ]:
train_dataset

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, BatchNormalization, ZeroPadding2D, MaxPooling2D
from keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop, Adam

from keras.applications.xception import Xception

## set up base model
base_model = Xception(weights = 'imagenet', include_top = False, input_shape = (IMG_SIZE, IMG_SIZE, 3))

## complete architecture
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(64, activation = 'relu', use_bias = True))
model.add(Dense(14, activation = 'sigmoid'))

## freeze base weights
base_model.trainable = False 

## compile 
LR_RT = 1e-4
model.compile(
    optimizer = RMSprop(learning_rate = LR_RT),
    loss = 'categorical_crossentropy',   
    metrics = ['accuracy']) 

83697664/83683744 [==============================] - 1s 0us/step


In [ ]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

MODEL_DIR = 'outputs'

reduce_lr = ReduceLROnPlateau(
    monitor = 'val_loss',
    factor = 0.2,
    patience = 3, 
    min_lr = 1e-7)

checkpoint = ModelCheckpoint(
    filepath = os.path.join(MODEL_DIR, '2_models/2_model_miranda_{epoch:02d}.h5'),
    monitor = 'val_acc',
    save_best_only = False,
    save_weights_only = False)

In [ ]:
EPOCHS = 10

start = time.time()
model.fit(
    train_dataset, 
    steps_per_epoch = train_dataset.n // BATCH_SIZE, 
    epochs = EPOCHS, 
    validation_data = valid_dataset,
    validation_steps = valid_dataset.n // BATCH_SIZE,
    callbacks = [reduce_lr, checkpoint]
)
end = time.time()
print(f'Running Time: {(end - start)/3600} hrs.')

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt

history = model.history
fig, ax = plt.subplots(1, 2, figsize = (15,5))
_= ax[0].set_title('Model Loss')
_= ax[0].plot(history.epoch, history.history['loss'], label = 'Train Loss')
_= ax[0].plot(history.epoch, history.history['val_loss'], label = 'Validation Loss')
_= ax[0].set_ylabel('Loss')
_= ax[0].set_xlabel('Epoch')
_= ax[0].legend()

_= ax[1].set_title('Model Accuracy')
_= ax[1].plot(history.epoch, history.history['accuracy'], label = 'Train Accuracy')
_= ax[1].plot(history.epoch, history.history['val_accuracy'], label = 'Validation Accuracy')
_= ax[1].set_ylabel('Accuracy')
_= ax[1].set_xlabel('Epoch')
_= ax[1].legend()
plt.show()